In [30]:
# Append System path for systematic imports
import sys
if (not('./' in sys.path)):
    sys.path.append('../')

In [31]:
#lets clean and transform the chat data
import pandas as pd
import os
filename="WhatsApp Chat with Pradyumn Y.txt"
data_path = '../data'
fpath_input = os.path.join(data_path,filename)
df=pd.read_csv(fpath_input,header=None,error_bad_lines=False,encoding='utf8')

C:\Users\arish\AppData\Local\Temp\ipykernel_135720\2809205461.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv(fpath_input,header=None,error_bad_lines=False,encoding='utf8')
Skipping line 447: expected 4 fields, saw 8
Skipping line 682: expected 4 fields, saw 13
Skipping line 1384: expected 4 fields, saw 6
Skipping line 1387: expected 4 fields, saw 7
Skipping line 1390: expected 4 fields, saw 7
Skipping line 1393: expected 4 fields, saw 7
Skipping line 1708: expected 4 fields, saw 12
Skipping line 1711: expected 4 fields, saw 12
Skipping line 4092: expected 4 fields, saw 5
Skipping line 4098: expected 4 fields, saw 6
Skipping line 6389: expected 4 fields, saw 5
Skipping line 6605: expected 4 fields, saw 5
Skipping line 7761: expected 4 fields, saw 7
Skipping line 8021: expected 4 fields, saw 5
Skipping line 8184: expected 4 fields, saw 7
Skipping line 8221: expected 4 fields

In [33]:
df= df.drop(0)
df.rename(columns={0:'Date',1:'Chat'},inplace=True)
Message= df["Chat"].str.split("-", n = 1, expand = True) 
df['Date']=df['Date'].str.replace(",","") 
df['Time']=Message[0]
df['Text']=Message[1]
Message1= df["Text"].str.split(":", n = 1, expand = True) 
df['Text']=Message1[1]
df['Name']=Message1[0]
df=df.drop(columns=['Chat'])
df['Text']=df['Text'].str.lower()
df['Text'] = df['Text'].str.replace('<media omitted>','MediaShared')
df['Text'] = df['Text'].str.replace('this message was deleted','DeletedMsg')    
df['Name'] = df['Name'].astype('str')

In [34]:
from collections import defaultdict
def convert_to_chat_covs(df):
    conv_start = defaultdict(lambda :[])
    conv_end = defaultdict(lambda :[])
    count = 0
    for i,j in df[['Text','Name']].values:
        if(j==' Pradyumn Y'):
            count+=1
            conv_start[count].append(i)
        else:
            conv_end[count].append(i)
    return conv_start,conv_end

In [35]:
conv_start,conv_end = convert_to_chat_covs(df)

In [36]:
conv_dicts = pd.DataFrame({'col_1':conv_start,'col_2':conv_end})

In [39]:
def join_as(x):
    try:
        if(len(x)!=1):
            return ''.join(x)
        else:
            return str(x[0])
    except:
        return 'nan'
conv_dicts.dropna(subset=['col_1','col_2'],inplace=True)
conv_dicts['response_text'] = conv_dicts['col_2'].apply(join_as)
conv_dicts['input_text'] = conv_dicts['col_1'].apply(join_as)

In [40]:
conv_dicts.head()

,col_1,col_2,response_text,input_text
1,[ aj kuch krna hai kya],[ aaja ghar],aaja ghar,aj kuch krna hai kya
2,[ deepanshu?],"[ chlega, ayga meko lene, gaadi nai hai mere...",chlega ayga meko lene gaadi nai hai mere paas,deepanshu?
3,[ pubg?],[ kal khelengr],kal khelengr,pubg?
5,[ 30 min bs],"[ thik hai aaja., ayga ?, ho gaya be ab, me...",thik hai aaja. ayga ? ho gaya be ab mein gaya...,30 min bs
7,[ mai ghum hi ra tha us area me],"[ tera phone nai, lag ra tha, bc, busy bata...",tera phone nai lag ra tha bc busy bata ra tha...,mai ghum hi ra tha us area me


In [42]:
conversation_data = conv_dicts[['input_text','response_text']]

In [47]:
conversation_data.to_csv(os.path.join(data_path,'conversation_data.csv'),index=False)

In [49]:
import pandas as pd
import re

def remove_hyperlinks(text):
    # Remove any URLs from the text using regex
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

def clean_whatsapp_data(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    df.dropna(inplace=True)
    df = df.rename(columns={'input_text':'question','response_text':'response'})
    df['question'] = df['question'].apply(lambda x:x.replace('"',''))
    df['response'] = df['response'].apply(lambda x:x.replace('"',''))

    # Assuming you have a 'Message' column in your CSV file
    # Remove hyperlinks from the 'Message' column
    df['question'] = df['question'].apply(remove_hyperlinks)
    df['response'] = df['response'].apply(remove_hyperlinks)

    # Filter out rows containing media content
    # Update the pattern if you have a different format
    media_pattern = r'<Media omitted>$|<Media omitted>$|image omitted$|video omitted$|GIF omitted$|sticker omitted$|audio omitted$|document omitted$'
    df = df[~df['question'].str.contains(media_pattern)]
    df = df[~df['response'].str.contains(media_pattern)]

    # Write the cleaned data to a new CSV file
#     df.to_csv(output_file, index=False)
    return df

# Usage example
input_csv = os.path.join(data_path,'conversation_data.csv')
# output_csv = 'cleaned_whatsapp_data.csv'
cleaned_df = clean_whatsapp_data(input_csv)


In [50]:
cleaned_df

,question,response
0,aj kuch krna hai kya,aaja ghar
1,deepanshu?,chlega ayga meko lene gaadi nai hai mere paas
2,pubg?,kal khelengr
3,30 min bs,thik hai aaja. ayga ? ho gaya be ab mein gaya...
4,mai ghum hi ra tha us area me,tera phone nai lag ra tha bc busy bata ra tha...
...,...,...
3789,bas,functional ke andar ivy vale frontend mein
3790,functional api hai,m toh sirf frontend likha hu baki ka i have n...
3791,bol bhai,aaja meet mein
3792,???,aana hackathon ka thoda karenge thoda ivy ka ...


In [52]:
cleaned_df['text_field'] = 'prady:'+cleaned_df['question']+'. aarish: '+cleaned_df['response']

In [56]:
cleaned_df[['text_field']].to_csv(os.path.join(data_path,'text_fields.csv'),index=False)